In [ ]:
%pip install -r requirements.txt

In [1]:
import os
import numpy as np
import time
import constructivo as constr
from auxiliar_functions import *
from local_search import *

#utilizar xlwt para escribir en excel


In [2]:
directory_path = '../JSSP_Instances'
directory_files = os.listdir(directory_path)
directory_files.sort()

In [3]:
import xlrd, xlwt
from xlutils.copy import copy as xl_copy
from xlwt import Workbook


def write_xlsx(machine_order, makespan, time, name_sheet, name_file):
    # Create a new workbook
    
    try:
        # open existing workbook
        rb = xlrd.open_workbook(name_file, formatting_info=True)
        # make a copy of it
        workbook = xl_copy(rb)
    except:
        workbook = Workbook()
    sheet = workbook.add_sheet(name_sheet)
    
    # Iterate over the matrix and write its values to the sheet
    for row_index, row in enumerate(machine_order, start=1):
        for col_index, value in enumerate(row, start=1):
            sheet.write(row_index - 1, col_index - 1, value)  # Note: row_index and col_index should start from 0
    
    # Write makespan and time to the next row
    row_index = len(machine_order)
    sheet.write(row_index, 0, int(makespan))
    sheet.write(row_index, 1, time)

    # Save the workbook to the specified file
    workbook.save(name_file)

In [4]:
with open('cotas.txt', 'r') as f:
    cotas = f.readlines()
    cotas = [int(line.strip()) for line in cotas]

with open('TimeLimit.txt', 'r') as f:
    # Note time is in minutes
    max_time = f.readlines()
    max_time = [int(line.strip()) for line in max_time]

In [6]:
#Reading all the information
for file_num in range(1, len(directory_files)+1):
  file = "JSSP" + str(file_num) + ".txt"
  with open(directory_path + "/" + file, "r" ) as f:
  #with open("../AAA_test", "r" ) as f:
    size = f.readline().split()
    # n = trabajos, m = maquinas
    n, m = int(size[0]), int(size[1])
    data = list(f)
    for i,line in enumerate(data):
      data[i] = [int(x) for x in line.strip().split() if x is not None and x != '']

    processing_time = data[0:n]
    machines_required = data[n:2*n]
  
  # Cota inferior
  machines = {i:[[ j+1, processing_time[j][z], z, False] for z in range(m) for j in range(n)  if machines_required[j][z] == i]for i in range(1,m+1)}
  cota_inferior = cotas[file_num-1]

  # Create job objects
  jobs_constr = [constr.Job(np.array(machines_required)[i, :], np.array(processing_time)[i, :], i + 1) for i in range(n)]

  #Sorted array in which the first value is always the first machine that 
  machines_constr = [constr.Machine(i+1) for i in range(m)]
  # Record the start time
  constr_start_time = time.time(); ans_constr, machine_order_constr = constr.constructivo(jobs_constr, machines_constr); constr_end_time = time.time()

  # Record the start time
  vns_start_time = time.time()
  vns_sol = variable_neighborhood_search(machine_order_constr, processing_time, machines_required, n, m, max_time=max_time[file_num-1])
  vns_end_time = time.time()

  multi_start_time = time.time()
  multi_sol = multi_search([machine_order_constr, vns_sol], processing_time, machines_required, n, m, method = interchange, noise_prob = 0.01, lns_destroy_perc=0.1, max_time=max_time[file_num-1])
  multi_end_time = time.time()

  print(file, "-> Constr: ", ans_constr, " Cota: ", cota_inferior)
  print("VNS -> " , makespan(vns_sol, processing_time, machines_required, n, m))
  print("multi_sol -> " , makespan(multi_sol, processing_time, machines_required, n, m))
  print()

  write_xlsx(vns_sol, makespan(vns_sol , processing_time, machines_required, n, m), vns_end_time - vns_start_time, file, "JSSP_coberndorm_vns.xlsx")
  write_xlsx(multi_sol, makespan(multi_sol, processing_time, machines_required, n, m), multi_end_time - multi_start_time, file, "JSSP_coberndorm_LNS_Noise_Multi_Sol.xlsx")


JSSP1.txt -> Constr:  1462  Cota:  991
VNS ->  1419
multi_sol ->  1403

JSSP2.txt -> Constr:  1446  Cota:  985
VNS ->  1428
multi_sol ->  1422

JSSP3.txt -> Constr:  1865  Cota:  1177
VNS ->  1845
multi_sol ->  1851

JSSP4.txt -> Constr:  1667  Cota:  1218
VNS ->  1608
multi_sol ->  1588

JSSP5.txt -> Constr:  2175  Cota:  1223
VNS ->  2082
multi_sol ->  2078

JSSP6.txt -> Constr:  1965  Cota:  1270
VNS ->  1962
multi_sol ->  1962

JSSP7.txt -> Constr:  2335  Cota:  1818
VNS ->  2291
multi_sol ->  2291

JSSP8.txt -> Constr:  2432  Cota:  1838
VNS ->  2392
multi_sol ->  2392

JSSP9.txt -> Constr:  2499  Cota:  1876
VNS ->  2405
multi_sol ->  2405

JSSP10.txt -> Constr:  2710  Cota:  1784
VNS ->  2607
multi_sol ->  2606

JSSP11.txt -> Constr:  3856  Cota:  2825
VNS ->  3507
multi_sol ->  3507

JSSP12.txt -> Constr:  3266  Cota:  3001
VNS ->  3231
multi_sol ->  3226

JSSP13.txt -> Constr:  3606  Cota:  2917
VNS ->  3547
multi_sol ->  3547

JSSP14.txt -> Constr:  3639  Cota:  2950
VNS ->  

In [5]:
import os
import xlrd

# Get a list of Excel files in the current directory
directory_files = os.listdir()
directory_files.sort()
directory_files = [file for file in directory_files if file.endswith('.xlsx')]

for file in directory_files:
    # Specify the path to your Excel file
    excel_file_path = file
    print("Current method: ", excel_file_path)
    average_distance_to_limit = []

    for i in range(1,16+1):
        sheet_name = f"JSSP{i}.txt"  # Change this to the desired sheet name
        column_letter = "B"      # Change this to the desired column letter

        # Open the Excel workbook
        workbook = xlrd.open_workbook(excel_file_path, formatting_info=True)

        # Select the specific sheet by name
        sheet = workbook.sheet_by_name(sheet_name)

        # Find the last row with data in the specified column (column B)
        max_row = sheet.nrows -1

        # Read the value from the last cell in column B
        makespan_val = sheet.cell_value(max_row, 0)
        # Read the value from the last cell in column B
        time_val = sheet.cell_value(max_row, 1)

        cota_file = cotas[i-1]

        efficacy = (makespan_val - cota_file)/cota_file
        average_distance_to_limit.append(efficacy)

        print(sheet_name)
        print(int(makespan_val))
        print(round(efficacy, 4))
        print()

    print("average distance to limit: ", round(sum(average_distance_to_limit)/len(average_distance_to_limit), 4))
    

Current method:  JSSP_coberndorm_LNS_Noise_Multi_Sol.xlsx
JSSP1.txt
1403
0.4157

JSSP2.txt
1422
0.4437

JSSP3.txt
1851
0.5726

JSSP4.txt
1588
0.3038

JSSP5.txt
2078
0.6991

JSSP6.txt
1962
0.5449

JSSP7.txt
2291
0.2602

JSSP8.txt
2392
0.3014

JSSP9.txt
2405
0.282

JSSP10.txt
2606
0.4608

JSSP11.txt
3507
0.2414

JSSP12.txt
3226
0.075

JSSP13.txt
3547
0.216

JSSP14.txt
3463
0.1739

JSSP15.txt
6135
0.1104

JSSP16.txt
5963
0.0506

average distance to limit:  0.322
Current method:  JSSP_coberndorm_vns.xlsx
JSSP1.txt
1419
0.4319

JSSP2.txt
1428
0.4497

JSSP3.txt
1845
0.5675

JSSP4.txt
1608
0.3202

JSSP5.txt
2082
0.7024

JSSP6.txt
1962
0.5449

JSSP7.txt
2291
0.2602

JSSP8.txt
2392
0.3014

JSSP9.txt
2405
0.282

JSSP10.txt
2607
0.4613

JSSP11.txt
3507
0.2414

JSSP12.txt
3231
0.0766

JSSP13.txt
3547
0.216

JSSP14.txt
3466
0.1749

JSSP15.txt
6154
0.1138

JSSP16.txt
5963
0.0506

average distance to limit:  0.3247


In [7]:
def read_sol(instance):
        sheet_name = f"JSSP{instance}.txt"  # Change this to the desired sheet name

        # Open the Excel workbook
        workbook = xlrd.open_workbook("/home/coberndorm/Documents/Semestre_IX/Heuristics/Trabajo_3/JSSP_coberndorm_vns.xlsx", formatting_info=True)

        # Select the specific sheet by name
        sheet = workbook.sheet_by_name(sheet_name)
        sol = []

        for i in range(sheet.nrows-1):
            sol.append([int(x) for x in sheet.row_values(i)])

        return sol

In [8]:
directory_path = '../JSSP_Instances'
directory_files = os.listdir(directory_path)
directory_files.sort()

In [12]:
for prob in [0.05, 0.2, 0.5]:
  efficacy = []
  for file_num in range(1, len(directory_files)+1):
    file = "JSSP" + str(file_num) + ".txt"
    with open(directory_path + "/" + file, "r" ) as f:
    #with open("../AAA_test", "r" ) as f:
      size = f.readline().split()
      # n = trabajos, m = maquinas
      n, m = int(size[0]), int(size[1])
      data = list(f)
      for i,line in enumerate(data):
        data[i] = [int(x) for x in line.strip().split() if x is not None and x != '']

      processing_time = data[0:n]
      machines_required = data[n:2*n]
    
    # Cota inferior
    machines = {i:[[ j+1, processing_time[j][z], z, False] for z in range(m) for j in range(n)  if machines_required[j][z] == i]for i in range(1,m+1)}
    cota_inferior = cotas[file_num-1]

    # Create job objects
    jobs_constr = [constr.Job(np.array(machines_required)[i, :], np.array(processing_time)[i, :], i + 1) for i in range(n)]

    #Sorted array in which the first value is always the first machine that 
    machines_constr = [constr.Machine(i+1) for i in range(m)]
    # Record the start time
    constr_start_time = time.time(); ans_constr, machine_order_constr = constr.constructivo(jobs_constr, machines_constr); constr_end_time = time.time()

    vns_sol = read_sol(file_num)

    multi_start_time = time.time()
    multi_sol = multi_search([machine_order_constr, vns_sol], processing_time, machines_required, n, m, method = interchange, noise_prob = 0.1, lns_destroy_perc=prob, max_time=max_time[file_num-1])
    multi_end_time = time.time()
    
    efficacy.append((makespan(multi_sol , processing_time, machines_required, n, m) - cota_inferior)/cota_inferior)

  print("efficacy of prob: ", prob, " -> ", round(sum(efficacy)/len(efficacy), 4))



efficacy of prob:  0.05  ->  0.3224
efficacy of prob:  0.2  ->  0.322
efficacy of prob:  0.5  ->  0.322
